# Исходные данные
## Структура датасета метаданных видео videos.parquet + features.parquet

| Поле | Описание |
|------|----------|
| video_id | Идентификатор видео |
| video_title | Название видео |
| channel_title | Название канала |
| v_channel_reg_datetime | Дата регистрации канала |
| v_channel_type | Тип канала |
| v_category | Категория видео |
| v_pub_datetime | Дата публикации видео |
| report_date | Дата на момент которой собирались данные признаки |
| total_comments | Количество комментариев под видео |
| v_year_views | Количество просмотров за год |
| v_month_views | Количество просмотров за месяц |
| v_week_views | Количество просмотров за неделю |
| v_day_views | Количество просмотров за день |
| v_likes | Количество лайков |
| v_dislikes | Количество дизлайков |
| v_duration | Продолжительность видео в миллисекундах |
| v_cr_click_like_7_days | Конверсия клика в лайк за 7 дней |
| v_cr_click_dislike_7_days | Конверсия клика в дизлайк за 7 дней |
| v_cr_click_vtop_7_days | Конверсия клика в нажатие "в топ" за 7 дней |
| v_cr_click_long_view_7_days | Конверсия клика в долгий просмотр за 7 дней |
| v_cr_click_comment_7_days | Конверсия клика в комментарий за 7 дней |
| v_cr_click_like_30_days | Конверсия клика в лайк за 30 дней |
| v_cr_click_dislike_30_days | Конверсия клика в дизлайк за 30 дней |
| v_cr_click_vtop_30_days | Конверсия клика в нажатие "в топ" за 30 дней |
| v_cr_click_long_view_30_days | Конверсия клика в долгий просмотр за 30 дней |
| v_cr_click_comment_30_days | Конверсия клика в комментарий за 30 дней |
| v_cr_click_like_1_days | Конверсия клика в лайк за 1 день |
| v_cr_click_dislike_1_days | Конверсия клика в дизлайк за 1 день |
| v_cr_click_vtop_1_days | Конверсия клика в нажатие "в топ" за 1 день |
| v_cr_click_long_view_1_days | Конверсия клика в долгий просмотр за 1 день |
| v_cr_click_comment_1_days | Конверсия клика в комментарий за 1 день |

# Вопросы

- Какие топ видео и каналы? Что смотрят в массе?
- Какая динамика? Что смотрят больше?

# Анализ данных

In [ ]:
import pandas as pd
import polars as pl
import nltk 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('popular')

In [ ]:
nltk.download('popular')

# Загрузка датасета
df = pd.read_parquet('rutube_case/automarkup_10k.parquet')

# Очистка текста (пример функции)
def clean_text(text):
    # Удаление специальных символов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    tokens = nltk.word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stopwords.words('russian')]
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

# Применение функции очистки текста к столбцу с запросами
df['cleaned_query'] = df['query'].apply(clean_text)

# Анализ частоты слов с использованием CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['cleaned_query'])

# Получение суммарной частоты слов
word_freq = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))

# Визуализация наиболее популярных слов с помощью Word Cloud
wordcloud = WordCloud(width=800, height=400, background_color ='white').generate_from_frequencies(word_freq)

# Отображение Word Cloud
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import pandas as pd
from collections import Counter

# Загрузка датасета
df = pd.read_parquet('rutube_case/automarkup.parquet')

# Нахождение самых частых запросов
query_counts = Counter(df['query'])
most_common_queries = query_counts.most_common(10)  # Топ-10 запросов

# Получение количества видео по самым частым запросам
video_count_by_query = df[df['query'].isin([q[0] for q in most_common_queries])].groupby('query')['video_id'].nunique()

# Нахождение видео, которые чаще всего запрашивают
most_requested_videos = df['video_id'].value_counts().head(10)  # Топ-10 запрашиваемых видео

# Распределение по авторизованности пользователей
auth_query_distribution = df.groupby('is_authorized')['query'].count()

# Анализ длины запросов
query_length = df['query'].apply(len)
query_length_stats = {
    'average_length': query_length.mean(),
    'max_length': query_length.max(),
    'min_length': query_length.min(),
}

# Классификация запросов и количество по категориям (примерная реализация)
# Для реальной классификации нужен более сложный подход, возможно с использованием машинного обучения
query_category = {
    'query': 'category'
    # Заполните категории в зависимости от запросов и предполагаемых категорий
}
df['category'] = df['query'].map(query_category)  # Маппинг запросов на категории
category_counts = df['category'].value_counts()

# Вывод результатов
print("Самые частые запросы:\n", most_common_queries)
print("\nКоличество уникальных видео по запросам:\n", video_count_by_query)
print("\nНаиболее часто запрашиваемые видео:\n", most_requested_videos)
print("\nРаспределение запросов по авторизованности пользователей:\n", auth_query_distribution)
print("\nСтатистика по длине запросов:\n", query_length_stats)
print("\nКоличество запросов по категориям:\n", category_counts)

# Визуализация данных (можно добавить графики, если это необходимо)


In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords

# Загрузка стоп-слов, для русского языка используйте 'russian'
stop_words = set(stopwords.words('russian'))

# Функция для очистки текста
def clean_text(text: pl.Series) -> pl.Series:
    # Удаление знаков препинания и цифр, приведение к нижнему регистру
    text = text.str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\d+', '', regex=True).str.to_lowercase()    
    # Удаление стоп-слов
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    return text

# Функция для очистки текста
def clean_text_pl(text: str) -> str:
    # Удаление знаков препинания и цифр, приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text).lower()
    # Удаление стоп-слов
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Функция для очистки текстовых столбцов в DataFrame
def clean_text_column(col: pl.Series) -> pl.Series:
    return col.map_elements(clean_text_pl)

# Загрузка датасетов
markup_df = pl.read_parquet('rutube_case/automarkup.parquet')
videos_df = pl.read_parquet('rutube_case/videos.parquet')
features_df = pl.read_parquet('rutube_case/features.parquet')

head_cnt = 30

# Соединение датасетов по полю video_id
merged_df = markup_df.join(videos_df, how='inner', on='video_id', )

merged_df = merged_df.with_columns([
    clean_text_column(pl.col('query')).alias('query_cleaned'),
    clean_text_column(pl.col('video_title')).alias('video_title_cleaned'),
    clean_text_column(pl.col('channel_title')).alias('channel_title_cleaned')
])


In [ ]:
# Количество каналов
channel_count = merged_df.select('channel_title').unique().count()

# Количество видео по каналам
video_count_by_channel = merged_df.groupby('channel_title').agg(pl.count('video_id').alias('video_count')).sort('video_count', reverse=True).head(head_cnt)

# Количество видео по категориям
video_count_by_category = merged_df.groupby('v_category').agg(pl.count('video_id').alias('video_count')).sort('video_count', reverse=True).head(head_cnt)

# Количество видео по реакциям пользователей
video_reaction_stats = merged_df.select(['v_likes', 'v_dislikes', 'total_comments']).sum()

# Визуализация количества видео по каналам
video_count_by_channel.to_pandas().set_index('channel_title')['video_count'].plot(kind='bar', title='Количество видео по каналам')
plt.show()

# Визуализация количества видео по категориям
video_count_by_category.to_pandas().set_index('v_category')['video_count'].plot(kind='bar', title='Количество видео по категориям')
plt.show()

# Визуализация количества видео по реакциям пользователей
video_reaction_stats.to_frame().plot(kind='bar', title='Реакции пользователей на видео')
plt.show()

In [ ]:
import polars as pl
df = pl.read_parquet('rutube_case/automarkup.parquet')
subset_df = df.head(10000)
subset_df.write_parquet('rutube_case/automarkup_10k.parquet')

